This notebook gives a simple example of how to use the ShaderMaterial to write custom shaders from the Python side. For further information about the shaders, consult the three.js docs.

In [1]:
#include <xthreejs/xthreejs.hpp>
#include <xwidgets/xlayout.hpp>
#include <xwidgets/xbox.hpp>
#include <xwidgets/xcolor_picker.hpp>
#include <xwidgets/xslider.hpp>

In [2]:
auto vertex_shader = R"(
uniform float time;
uniform vec2 resolution;

void main() {
    vec3 pos = vec3(position.x + time * resolution.x, position.y + time * resolution.y, position.z);
    gl_Position = projectionMatrix * modelViewMatrix * vec4( pos, 1.0 );
}
)";

In [3]:
auto fragment_shader = R"(
uniform vec3 user_color;

void main() {
    #ifdef FIX_COLOR
    gl_FragColor = vec4(0.5, 0, 0, 1.0);
    #else
    gl_FragColor = vec4(user_color, 1.0);
    #endif
}
)";

In [4]:
auto uniform = xthree::xuniform_lib["lights"];
uniform["time"] = xeus::xjson::object({{"value", 0}});
uniform["resolution"] = xeus::xjson::object({{"value", {1, 1}}});
uniform["user_color"] = xeus::xjson::object({{"value", "green"}});

In [5]:
auto defines = R"(
{
    "FIX_COLOR": 1
}
)"_json;

In [6]:
auto material = xthree::shader_material_generator()
    .uniforms(uniform)
    .defines(defines)
    .vertexShader(vertex_shader)
    .fragmentShader(fragment_shader)
    .lights(true)
    .finalize();

In [7]:
material

A Jupyter widget

In [8]:
material.defines = xeus::xjson::object();
material.update();

In [9]:
// Uniform editors:
auto picker = xw::color_picker_generator()
    .value(material.uniforms()["user_color"]["value"])
    .finalize();

auto time_slider = xw::slider_generator<double>()
    .value(material.uniforms()["time"]["value"])
    .min(-15)
    .max(15)
    .finalize();

In [10]:
XOBSERVE(picker, value, [&](const auto& p){
    auto uniform = material.uniforms();
    uniform["user_color"]["value"] = p.value();
    material.uniforms = uniform;
    material.update();
});

XOBSERVE(time_slider, value, [&](const auto& s){
    auto uniform = material.uniforms();
    uniform["time"]["value"] = s.value();
    material.uniforms = uniform;
    material.update();
});

// Present "dashboard" in VBox:
auto vbox = xw::vbox();
vbox.add(xthree::preview(material));
vbox.add(picker);
vbox.add(time_slider);

In [11]:
vbox

A Jupyter widget